In [2]:
import pandas as pd
import numpy as np
import z3
from tqdm import tqdm

In [3]:
z3.set_option("parallel.enable", True)
z3.set_option("parallel.threads.max", 1024)

In [4]:
NUM_FLOW = 128
DATA_NAME = "single0"
TOPO_NAME = "0"
task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:NUM_FLOW]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)



## 1. Model

In [5]:
NUM_SW = 8
NUM_ES = 8
NUM_NODE = NUM_ES + NUM_SW
NETWORK_ERROR = 0
NUM_WINDOW = 20

LCM = int(np.lcm.reduce(task['period']))
net = np.zeros(shape = (NUM_NODE, NUM_NODE))

In [6]:
NG = 10

Schedule table

In [7]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['N'] = row['q_num']
    net_var[row['link']]['s'] = row['rate']
    net_var[row['link']]['tproc'] = row['t_proc']
    net_var[row['link']]['t'] = z3.IntVector('t_%s'%row['link'], NUM_WINDOW)
    net_var[row['link']]['v'] = z3.IntVector('v_%s'%row['link'], NUM_WINDOW)
    net_var[row['link']]['c'] = z3.IntVector('c_%s'%row['link'], NUM_WINDOW)
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

Task model

In [8]:
task_var = {}
task_attr = {}
packet_weight = {}

In [9]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [10]:
for i, row in task.iterrows():
    task.loc[i,'route'] = str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0]))))
    task_var.setdefault(i, {})
    task_attr.setdefault(i, {})
    route = eval(task.loc[i, 'route'])
    
    task_attr[i]['C'] = int(row['size'])
    task_attr[i]['T'] = int(row['period']) 
    task_attr[i]['L'] = int(row['deadline'])
    task_attr[i]['rho'] = (int(task_attr[i]['C'] * 8) + sync_error) * (len(route) - 1) / task_attr[i]['L']
#     task_attr[i]['J'] = 1000000000 
    task_attr[i]['d'] = []
    for j in range(int(LCM / task_attr[i]['T'])):
        packet_weight[(i,j)] = task_attr[i]['L'] + j * task_attr[i]['T']
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})
        task_var[i][link] = {}
        task_var[i][link]['alpha'] = []
        task_var[i][link]['w'] = []
        task_var[i][link]['group'] = []
        for j in range(int(LCM / task_attr[i]['T'])):
            task_var[i][link]['alpha'].append(z3.Int('alpha' +  str(i) + '_' + str(link) + '_' + str(j)))
            task_var[i][link]['w'].append(z3.Int('w' +  str(i) + '_' + str(link) + '_' + str(j)))
            task_var[i][link]['group'].append(None)
        task_attr[i].setdefault(link, {})
        task_attr[i][link]['l'] = int(task_attr[i]['C']) * 8 
        
        
#         task_var[i][link]['phi'] = []
#         task_var[i][link]['p'] =  z3.Int('p_' +  str(i) + '_' + str(link))
#         task_var[i][link]['T'] = np.ceil(row['period'] / net_var[str(link)]['mt'])
#         task_var[i][link]['L'] = np.ceil((row['size'] * 8 / net_var[str(link)]['s']) / net_var[str(link)]['mt'])
#         for j in range(0, int(LCM / row['period'])):
#              task_var[i][link]['phi'].append(z3.Int('phi_' +  str(i) + '_' + str(link) + '_' + str(j)))

## 2. Queue assignment

In [11]:
phat = {}
for link in net_var:
    phat.setdefault(link, [0] * 8)

for f, attr in sorted(task_attr.items(), key = lambda x:x[1]['rho'], reverse=True):
    min_h = -1
    min_value = np.inf
    for g in range(8):
        result = max([phat[link][g] for link in list(task_var[f].keys())])
        if result < min_value:
            min_h = g
            min_value = result
    
    for link in list(task_var[f].keys()):
        phat[link][min_h] += task_attr[f]['rho']
        task_attr[f][link]['q'] = min_h

## 3. Taskset decomposition

In [12]:
packet_group = {}

In [13]:
packet_weight = [x[0] for x in sorted(packet_weight.items(), key = lambda x:x[1])]

In [14]:
group_size = int(np.ceil(len(packet_weight) / NG))

In [15]:
packet_group = [packet_weight[i * group_size: (i + 1) * group_size] for i in range((len(packet_weight) + group_size - 1) // group_size )]

In [16]:
for inte, group in enumerate(packet_group):
    for i, ins in group:
        for link in task_var[i]:
            task_var[i][link]['group'][ins] = inte

## 4. SMT model

Range constraints

\begin{aligned}
&\forall n_i \in W, \quad \forall y \in[2, T], t_{i, 1}=0 \\
&\quad\left(\left(t_{i, y}=0\right) \wedge \bigwedge_{\forall z \in[y+1, T]}\left(t_{i, z}=0\right)\right) \\
&\quad \vee\left(\left(t_{i, y}>0\right) \wedge\left(1 \leq t_{i, y-1}<t_{i, y} \leq H\right)\right) \\
&\forall n_i \in W, \quad \forall y \in[1, T], \quad \forall r \in[1, Q], c_{i, y, r} \in\{0, r\} \\
&\forall n_i \in W, \quad \forall y \in[1, T], \quad \forall j \in[1, V], v_{i, y, j} \in\{0,1\} . \\
&\forall f_a \in F, \quad 1 \leq q_a \leq Q . \\
&\forall f_a \in F, \quad \forall b \in\left[1,\left|\Pi_a\right|\right), \forall k \in\left[0, \frac{H}{p_a}\right) \\
&k \times p_a+1 \leq \alpha_{a, b}^k<\beta_{a, b}^k \leq d_a+k \times p_a \\
&\beta_{a, b}^k-\alpha_{a, b}^k=\gamma_a^{+} .
\end{aligned}

In [ ]:
interation = 0
s = z3.Optimize()
s.set("timeout", time_out * 1000)
# s.set("timeout", 4 * 1000) # 4s

In [ ]:
for link in net_var:
    s.add(
        net_var[link]['t'][0] < LCM,
        net_var[link]['t'][0] >= 0,
        # z3.If(
        #         net_var[link]['t'][0] == 0,
        #         z3.Sum([net_var[link]['t'][i] for i in range(NUM_WINDOW)]) == 0,
        #         True
        #     )
    )
    for y in range(1, NUM_WINDOW):
        s.add(
            net_var[link]['t'][y] < LCM,
            net_var[link]['t'][y] >= 0,
            net_var[link]['t'][y] >= net_var[link]['t'][y - 1]
            # z3.Or(
            #     z3.Sum([net_var[link]['t'][i] for i in range(y, NUM_WINDOW)]) == 0,
            #     z3.And(
            #         net_var[link]['t'][y] > 0,
            #         net_var[link]['t'][y] > net_var[link]['t'][y - 1]
            #     )
            # )
        )
    for y in range(NUM_WINDOW):
        s.add(
            net_var[link]['c'][y] >= 0,
            net_var[link]['c'][y] < net_var[link]['N']
        )
        # s.add(
        #     net_var[link]['v'][y] >= 0,
        #     net_var[link]['v'][y] <= 1
        # )
    for y in range(NUM_WINDOW):
        s.add(
            net_var[link]['v'][y] >= 0,
            net_var[link]['v'][y] <= 1
        )
        

In [ ]:
for i in task_var:
    for link in task_var[i]:
        for k, ins in enumerate(task_var[i][link]['alpha']):
            if task_var[i][link]['group'][k] == interation:
                s.add(
                   k * (task_attr[i]['T']) <= ins,
                   ins <= (k + 1) * (task_attr[i]['T'])
                )
        for k, ins in enumerate(task_var[i][link]['w']):
            if task_var[i][link]['group'][k] == interation:
                s.add(
                   0 <= ins,
                   ins < NUM_WINDOW
                )

Path sequence constraint

\begin{equation}
\forall f_a \in F, \quad \forall b \in\left[1,\left|\Pi_a\right|-1\right), \quad \forall k \in\left[0, \frac{H}{p_a}\right), \alpha_{a, b+1}^k>\beta_{a, b}^k
\end{equation}

In [ ]:
for i in task_var:
    links = list(task_var[i].keys())
    for hop, link in enumerate(links[:-1]):
        for k, ins in enumerate(task_var[i][link]['alpha']):
            if task_var[i][link]['group'][k] == interation:
                s.add(
                    ins + task_attr[i][link]['l'] + net_var[link]['tproc'] <= task_var[i][links[hop + 1]]['alpha'][k]
                )

Real-time constraint

\begin{equation}
\forall f_a \in F, \quad \forall k \in\left[0, \frac{H}{p_a}\right), \beta_{a,\left|\Pi_a\right|-1}^k \leq d_a+k \times p_a .
\end{equation}

In [ ]:
for i in task_var:
    s_hop = list(task_var[i].keys())[0]
    e_hop = list(task_var[i].keys())[-1]
    for k, ins in enumerate(task_var[i][e_hop]['alpha']):
        if task_var[i][s_hop]['group'][k] == interation:
            s.add(
                ins + task_attr[i][e_hop]['l'] + net_var[e_hop]['tproc'] <= task_var[i][s_hop]['alpha'][k] + task_attr[i]['L']
            )

Link conﬂict constraint

\begin{equation}
\begin{aligned}
\forall f_a, f_g \in F, a \neq g, \quad \forall b \in\left[1,\left|\Pi_a\right|\right), \quad \forall h \in\left[1,\left|\Pi_g\right|\right), \\
& \forall k \in\left[0, \frac{H}{p_a}\right), \forall m \in\left[0, \frac{H}{p_g}\right), \\
& \neg\left(\pi_{a, b}=\pi_{g, h}\right) \vee \neg\left(\pi_{a, b+1}=\pi_{g, h+1}\right) \\
& \vee\left(\alpha_{a, b}^k>\beta_{g, h}^m\right) \vee\left(\alpha_{g, h}^m>\beta_{a, b}^k\right) .
\end{aligned}
\end{equation}

In [ ]:
for link in net_var:
    for fa, fg in [(fa, fg) for fa in task_attr for fg in task_attr if fg > fa and link in task_var[fa] and link in task_var[fg]]:
        for k, m in [(k, m) for k in range(len(task_var[fa][link]['alpha'])) for m in range(len(task_var[fg][link]['alpha']))]:
            if task_var[fa][link]['group'][k] == interation and task_var[fg][link]['group'][m] == interation:
                ins_k = task_var[fa][link]['alpha'][k]
                ins_m = task_var[fg][link]['alpha'][m]
                s.add(
                    z3.Or(
                        ins_k >= ins_m + task_attr[fa][link]['l'],
                        ins_m >= ins_k + task_attr[fg][link]['l']
                    )
                )
print(s.check())

Queue conﬂict constraint

$\forall f_a, f_g \in F, a \neq g, \quad \forall b \in\left[2,\left|\Pi_a\right|\right), \quad \forall h \in\left[2,\left|\Pi_g\right|\right)$
$$
\begin{array}{l}
\forall k \in\left[0, \frac{H}{p_a}\right), \quad \forall m \in\left[0, \frac{H}{p_g}\right), \\
\neg\left(\pi_{a, b}=\pi_{g, h}\right) \vee \neg\left(\pi_{a, b+1}=\pi_{g, h+1}\right) \vee \neg\left(q_a=q_g\right) \\
\vee\left(\beta_{a, b}^k<\alpha_{g, h-1}^m\right) \vee\left(\beta_{g, h}^m<\alpha_{a, b-1}^k\right) .
\end{array}
$$

In [ ]:
for fa, fg in [(fa, fg) for fa in task_attr for fg in task_attr if fg > fa]:
    path_i = list(task_var[fa].keys())
    path_j = list(task_var[fg].keys())
    for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b)
                    for _x, i_a_b in enumerate(path_i) 
                    for _y, j_a_b in enumerate(path_j) 
                    if i_a_b == j_a_b and task_attr[fa][i_a_b]['q'] == task_attr[fa][j_a_b]['q']]:
        for k, m in [(k, m) for k in range(len(task_var[fa][x_a]['alpha'])) for m in range(len(task_var[fg][y_a]['alpha']))]:
            if task_var[fa][x_a]['group'][k] == interation and task_var[fg][y_a]['group'][m] == interation:
                s.add(
                    z3.Or(
                        task_var[fa][x_a]['alpha'][k] + task_attr[fa][x_a]['l'] + net_var[x_a]['tproc'] >= task_var[fg][a_b]['alpha'][m],
                        task_var[fg][y_a]['alpha'][m] + task_attr[fg][y_a]['l'] + net_var[y_a]['tproc'] >= task_var[fa][a_b]['alpha'][k]
                    )

                )

Time validity constraint

\begin{equation}
\begin{array}{l}
\forall f_a \in F, \quad \forall b \in\left[2,\left|\Pi_a\right|\right), \operatorname{port}\left(f_a, \pi_{a, b}\right)=n_{i, j}\\
\forall k \in\left[0, \frac{H}{p_a}\right), \quad \bigvee_{\forall g \in\left[k \times p_a+1, k \times p_a+d_a\right]}\left(\left(\alpha_{a, b}^k=g\right) \wedge A(g)\right),
\end{array}
\end{equation}

\begin{equation}
\begin{aligned}
A(x)=((x>&\left.\left.\beta_{a, b-1}^k\right) \wedge B(x)\right) \\
& \wedge \bigwedge_{\forall h \in\left[k \times p_a+1, x\right)}\left(\left(h \leq \beta_{a, b-1}^k\right) \vee \neg B(h)\right)
\end{aligned}
\end{equation}

\begin{equation}
\begin{aligned}
B(x)=& \bigvee_{\forall e_{i, y} \in E_i}\left(( t _ { i , y } = 0 ) \vee \left(v_{i, y, j} \wedge \bigvee_{\forall r \in[1, Q]}\left(c_{i, y, r}=q_a\right)\right.\right.\\
& \wedge \bigwedge_{\forall e_{i, z} \in E_i, z \neq y}\left(\left(t_{i, z}=0\right) \vee \neg v_{i, z, j} \vee\left(t_{i, z}<t_{i, y} \leq x\right)\right.\\
&\left.\left.\left.\vee\left(t_{i, z}>x\right)\right)\right)\right)
\end{aligned}
\end{equation}

In [ ]:
for i in task_var:
    links = list(task_var[i].keys())
    for hop, link in enumerate(links):
        last_link = links[hop - 1]
        for ins in range(len(task_var[i][link]['alpha'])):
            if task_var[i][link]['group'][ins] == interation:
                s.add(
                    z3.Or([
                        ## One widnow is opened for the task
                        z3.And(
                        net_var[link]['t'][x] <= task_var[i][link]['alpha'][ins],
                        net_var[link]['t'][x + 1] >= task_var[i][link]['alpha'][ins] + task_attr[i][link]['l'],
                        net_var[link]['c'][x] == task_attr[i][link]['q'],
                        net_var[link]['c'][x + 1] != net_var[link]['c'][x],
                        net_var[link]['v'][x] == 1,
                        task_var[i][link]['w'][ins] == x, ## Make sure it can only from this window
                        ## At least meet one of the following conditions:
                        ## (1) alpha = t: start from the beginning of the window
                        ## (2) or alpha = beta_{-1}: just arrive from last hop
                        ## (3) or alpha = other beta: start hehind other traffics ->
                        ## We can get benefits from only searching the frames with same queue and same link to speed up (3)
                        z3.Or(
                            net_var[link]['t'][x] == task_var[i][link]['alpha'][ins],
                            task_var[i][last_link]['alpha'][ins] + task_attr[i][last_link]['l'] + net_var[link]['tproc']
                            == task_var[i][link]['alpha'][ins],
                            z3.Or(
                                [task_var[j][link]['alpha'][l] + task_attr[j][link]['l'] == task_var[i][link]['alpha'][ins]
                                for j in task_var if link in task_var[j] and task_attr[j][link]['q'] == task_attr[i][link]['q']
                                for l in range(int(LCM / task_attr[j]['T'])) ]
                            )
                        ) if hop != 0 else True
                    )
                    for x in range(NUM_WINDOW - 1)])
                )
                

In [ ]:
# upper_bound = z3.Int('UP')
# for link in net_var:
#     s.add(
#         upper_bound >= net_var[link]['t'][-1]
#     )

# s.minimize(
#     upper_bound
# )

# upper_bound = z3.Int('UP')
# for link in net_var:
#     s.add(
#         upper_bound >= net_var[link]['t'][-1]
#     )

# s.minimize(
#     z3.Sum(
#         [z3.Sum(net_var[link]['v']) for link in net_var]
#     )
# )

s.minimize(
    z3.Sum(
        [net_var[link]['t'][-1] for link in net_var]
    )
)

In [ ]:
print(s.check())
result = s.model()

## Output Schedule

In [ ]:
GCL = []

for link in net_var:
    for t in range(NUM_WINDOW):
        if 
    result[net_var[link]['t'][t]].as_long(), result[net_var[link]['c'][t]].as_long()) for t in range(NUM_WINDOW)])

In [ ]:
for i in task_attr:
    for link in task_var[i]:
        if task_var[i][link]['group'][0] == 0:
            print(i, link, task_attr[i][link]['l'], task_attr[i]['T'], task_attr[i][link]['q'])

In [ ]:
for iteration, result in enumerate(group_result):
    for i in task_var:
        for link in task_var[i]:
            for k in range(len(task_var[i][link]['alpha'])):
                if task_var[i][link]['group'][k] == iteration:
                    print(i, link, result[task_var[i][link]['alpha'][k]] * macrotick, result[task_var[i][link]['w'][k]])

In [ ]:
# for i in task_var:
#     for link in task_var[i]:
#         if task_var[i][link]['group'][0] == 0:
#             print(i, link, result[task_var[i][link]['alpha'][0]])
#         break

In [ ]:
for link in net_var:
    print(link, [(result[net_var[link]['t'][t]].as_long(), result[net_var[link]['c'][t]].as_long()) for t in range(NUM_WINDOW)])

In [17]:
group_result = []

for interation in range(NG):
    ## Reset the net_var
    
    for _, row in network.iterrows():
        net_var.setdefault(row['link'], {})
        net_var[row['link']]['N'] = row['q_num']
        net_var[row['link']]['s'] = row['rate']
        net_var[row['link']]['tproc'] = row['t_proc']
        net_var[row['link']]['t'] = z3.IntVector('t_%s'%row['link'], NUM_WINDOW)
        net_var[row['link']]['v'] = z3.IntVector('v_%s'%row['link'], NUM_WINDOW)
        net_var[row['link']]['c'] = z3.IntVector('c_%s'%row['link'], NUM_WINDOW)
        if interation == 0:
            net_var[row['link']]['max_c'] = 0
            net_var[row['link']]['max_q'] = 0
    
    s = z3.Optimize()
    s.set("timeout", time_out * 1000)
    for link in net_var:

        for y in range(NUM_WINDOW):
            s.add(
                net_var[link]['t'][y] < LCM,
                net_var[link]['t'][y] >= net_var[link]['max_c'],
                net_var[link]['t'][y] >= net_var[link]['t'][y - 1] if y > 0 else True
            )
        for y in range(NUM_WINDOW):
            s.add(
                net_var[link]['c'][y] >= 0,
                net_var[link]['c'][y] < net_var[link]['N']
            )
        for y in range(NUM_WINDOW):
            s.add(
            net_var[link]['v'][y] >= 0,
            net_var[link]['v'][y] <= 1,
            # net_var[link]['v'][y] <= net_var[link]['v'][y - 1] if y != 0 else True,
            # z3.Or(
            #     net_var[link]['v'][y] == 0,
            #     net_var[link]['t'][y + 1] - net_var[link]['t'][y] > 0
            # ) if y != NUM_WINDOW - 1 else True
            )
        


    for i in task_var:
        for link in task_var[i]:
            for k, ins in enumerate(task_var[i][link]['alpha']):
                if task_var[i][link]['group'][k] == interation:
                    s.add(
                       k * (task_attr[i]['T']) <= ins,
                       ins <= (k + 1) * (task_attr[i]['T'])
                    )
            for k, ins in enumerate(task_var[i][link]['w']):
                if task_var[i][link]['group'][k] == interation:
                    s.add(
                       0 <= ins,
                       ins < NUM_WINDOW
                    )

    for i in task_var:
        links = list(task_var[i].keys())
        for hop, link in enumerate(links[:-1]):
            for k, ins in enumerate(task_var[i][link]['alpha']):
                if task_var[i][link]['group'][k] == interation:
                    s.add(
                        ins + task_attr[i][link]['l'] + net_var[link]['tproc'] <= task_var[i][links[hop + 1]]['alpha'][k]
                    )

    for i in task_var:
        s_hop = list(task_var[i].keys())[0]
        e_hop = list(task_var[i].keys())[-1]
        for k, ins in enumerate(task_var[i][e_hop]['alpha']):
            if task_var[i][s_hop]['group'][k] == interation:
                s.add(
                    ins + task_attr[i][e_hop]['l'] + net_var[e_hop]['tproc'] <= task_var[i][s_hop]['alpha'][k] + task_attr[i]['L']
                )


    for link in net_var:
        for fa, fg in [(fa, fg) for fa in task_attr for fg in task_attr if fg > fa and link in task_var[fa] and link in task_var[fg]]:
            for k, m in [(k, m) for k in range(len(task_var[fa][link]['alpha'])) for m in range(len(task_var[fg][link]['alpha']))]:
                if task_var[fa][link]['group'][k] == interation and task_var[fg][link]['group'][m] == interation:
                    ins_k = task_var[fa][link]['alpha'][k]
                    ins_m = task_var[fg][link]['alpha'][m]
                    s.add(
                        z3.Or(
                            ins_k >= ins_m + task_attr[fa][link]['l'],
                            ins_m >= ins_k + task_attr[fg][link]['l']
                        )
                    )


    for fa, fg in [(fa, fg) for fa in task_attr for fg in task_attr if fg > fa]:
        path_i = list(task_var[fa].keys())
        path_j = list(task_var[fg].keys())
        for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b)
                        for _x, i_a_b in enumerate(path_i) 
                        for _y, j_a_b in enumerate(path_j) 
                        if i_a_b == j_a_b and task_attr[fa][i_a_b]['q'] == task_attr[fa][j_a_b]['q']]:
            for k, m in [(k, m) for k in range(len(task_var[fa][x_a]['alpha'])) for m in range(len(task_var[fg][y_a]['alpha']))]:
                if task_var[fa][x_a]['group'][k] == interation and task_var[fg][y_a]['group'][m] == interation:
                    s.add(
                        z3.Or(
                            task_var[fa][x_a]['alpha'][k] + task_attr[fa][x_a]['l'] + net_var[x_a]['tproc'] >= task_var[fg][a_b]['alpha'][m],
                            task_var[fg][y_a]['alpha'][m] + task_attr[fg][y_a]['l'] + net_var[y_a]['tproc'] >= task_var[fa][a_b]['alpha'][k]
                        )
                    )
            

    for i in task_var:
        links = list(task_var[i].keys())
        for hop, link in enumerate(links):
            last_link = links[hop - 1] if hop != 0 else None
            for ins in range(len(task_var[i][link]['alpha'])):
                if task_var[i][link]['group'][ins] == interation:
                    s.add(
                        z3.Or([
                            ## One widnow is opened for the task
                            z3.And(
                            net_var[link]['t'][x] <= task_var[i][link]['alpha'][ins],
                            net_var[link]['t'][x + 1] >= task_var[i][link]['alpha'][ins] + task_attr[i][link]['l'],
                            net_var[link]['c'][x] == task_attr[i][link]['q'],
                            net_var[link]['c'][x + 1] != net_var[link]['c'][x],
                            # net_var[link]['c'][x] != net_var[link]['max_q'] if x == 0 else True,
                            net_var[link]['v'][x] == 1,
                            task_var[i][link]['w'][ins] == x, ## Make sure one frame can only use one window
                            ## At least meet one of the following conditions:
                            ## (1) alpha = t: start from the beginning of the window
                            ## (2) or alpha = beta_{-1}: just arrive from last hop
                            ## (3) or alpha = other beta: start hehind other traffics ->
                            ## We can get benefits from only searching the frames with same queue and same link to speed up (3)
                            z3.Or(
                                net_var[link]['t'][x] == task_var[i][link]['alpha'][ins],
                                task_var[i][last_link]['alpha'][ins] + task_attr[i][last_link]['l'] + net_var[link]['tproc']
                                == task_var[i][link]['alpha'][ins] if hop != 0 else True,
                                z3.Or(
                                    [task_var[j][link]['alpha'][l] + task_attr[j][link]['l'] == task_var[i][link]['alpha'][ins]
                                    for j in task_var if link in task_var[j] and task_attr[j][link]['q'] == task_attr[i][link]['q']
                                    for l in range(int(LCM / task_attr[j]['T'])) ]
                                )
                            ) 
                        )
                        for x in range(NUM_WINDOW - 1)])
                    )
                
    upper_bound = z3.Int('UP')
    for link in net_var:
        s.add(
            upper_bound >= net_var[link]['t'][-1]
        )

    s.minimize(
        upper_bound
    )

    print(interation, s.check())
    
    result = s.model()

    for link in net_var:
        net_var[link]['max_c'] = result[net_var[link]['t'][-1]].as_long()
        # net_var[link]['max_c'] =  max([result[net_var[link]['t'][-1]].as_long() for link in net_var])
        for x in range(NUM_WINDOW - 1):
            if result[net_var[link]['t'][x + 1]].as_long() - result[net_var[link]['t'][x]].as_long() > 0:
                net_var[link]['max_q'] = result[net_var[link]['c'][x]].as_long()

    group_result.append(result)

0 sat
1 sat
2 sat
3 sat
4 sat
5 sat
6 sat
7 sat
8 sat
9 sat


In [24]:
GCL = []
for i, result in enumerate(group_result):
    for link in net_var:
        for t in range(NUM_WINDOW - 1):
            if result[net_var[link]['t'][t+1]].as_long() - result[net_var[link]['t'][t]].as_long() > 0:
                start = result[net_var[link]['t'][t]].as_long()
                end = result[net_var[link]['t'][t + 1]].as_long()
                queue = result[net_var[link]['c'][t]].as_long()
                GCL.append(
                    [eval(link), queue, (start) * macrotick, (end) * macrotick, LCM * macrotick]
                )
            

In [19]:
## Offset
OFFSET = []
for interation, result in enumerate(group_result):
    for i in task_var:
        link = list(task_var[i].keys())[0]
        for ins_id in range(len(task_var[i][link]['alpha'])):
            if task_var[i][link]['group'][ins_id] == interation:
                offset = result[task_var[i][link]['alpha'][ins_id]].as_long()
                OFFSET.append([i, ins_id, (task_attr[i]['T'] - offset) * macrotick])

In [20]:
ROUTE = []
for i, row in task.iterrows():
    route = list(task_var[i].keys())
    for h, v in enumerate(route):
        ROUTE.append(
            [i, eval(v)]
        )

In [21]:
QUEUE = []
for i in task_var:
    for e in task_var[i]:
        for ins_id in range(len(task_var[i][e]['alpha'])):
            queue = task_attr[i][e]['q']
            QUEUE.append([i, ins_id, eval(e), queue])

In [22]:
GCL = pd.DataFrame(GCL)
GCL.columns = ["link", "queue", "start", "end", "cycle"]
GCL.to_csv("ACCESS2020-%s-%d-%s-GCL.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

OFFSET = pd.DataFrame(OFFSET)
OFFSET.columns = ['id', 'ins_id', 'offset']
OFFSET.to_csv("ACCESS2020-%s-%d-%s-OFFSET.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

ROUTE = pd.DataFrame(ROUTE)
ROUTE.columns = ['id', 'link']
ROUTE.to_csv("ACCESS2020-%s-%d-%s-ROUTE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)

QUEUE = pd.DataFrame(QUEUE)
QUEUE.columns = ['id','ins_id','link','queue']
QUEUE.to_csv("ACCESS2020-%s-%d-%s-QUEUE.csv"%(DATA_NAME,NUM_FLOW,TOPO_NAME), index=False)